# CSP Coding Assignment: Sudoku Solver

## Introduction
In this assignment, you will extend the Sudoku-solving agent developed in the classroom lectures to solve diagonal Sudoku puzzles. A diagonal Sudoku puzzle is identical to traditional Sudoku puzzles with the added constraint that the boxes on the two main diagonals of the board must also contain the digits 1-9 in each cell (just like the rows, columns, and 3x3 blocks). You will also implement another strategy called "Naked Pairs", described [here](https://www.learn-sudoku.com/naked-pairs.html)<br>
<img style="float: center;height:350px;" src="naked-twins.png"><br>

In [184]:
def cross(a, b):
      return [s+t for s in a for t in b]

In [185]:
rows = 'ABCDEFGHI'
cols = '123456789'
boxes = cross(rows, cols)

In [186]:
row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

# TODO: creat the diagonal units of the boards
diagonal_units = [[rows[i]+cols[i] for i in range(len(rows))], [rows[i]+cols[-i-1] for i in range(len(rows))]]

[['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9'], ['A9', 'B8', 'C7', 'D6', 'E5', 'F4', 'G3', 'H2', 'I1']]


In [187]:
# TODO: Update the unit list to add the new diagonal units
unitlist = row_units + column_units + square_units + diagonal_units

In [188]:
# Must be called after all units (including diagonals) are added to the unitlist
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [189]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [190]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    dict = {}
    for box in boxes:
        dict[box] = grid[boxes.index(box)]
    for key in dict.keys():
        if dict[key] == '.':
            dict[key] = '123456789'
    return dict

In [191]:
def eliminate(values):
    """Apply the eliminate strategy to a Sudoku puzzle

    The eliminate strategy says that if a box has a value assigned, then none
    of the peers of that box can have the same value.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the assigned values eliminated from peers
    """
    for box in boxes:
        if len(values[box]) == 1:
            for peer in peers[box]:
                values[peer] = values[peer].replace(values[box], '')
    return values

In [192]:
def only_choice(values):
    """Apply the only choice strategy to a Sudoku puzzle

    The only choice strategy says that if only one box in a unit allows a certain
    digit, then that box must be assigned that digit.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with all single-valued boxes assigned 
    """
    #look at every zone
    #if there is a number that only appears once in that zone, then assign that number to that box
    for unit in unitlist:
        for digit in cols:
            digit_places = [box for box in unit if digit in values[box]]
            if len(digit_places) == 1:
                values[digit_places[0]] = digit
    return values

In [193]:
def naked_pairs(values):
    """Eliminate values using the naked pairs strategy.

    The naked pairs strategy says that if you have two or more unallocated boxes
    in a unit and there are only two digits that can go in those two boxes, then
    those two digits can be eliminated from the possible assignments of all other
    boxes in the same unit.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the naked pairs eliminated from peers      
    """
    
    for unit in unitlist:
        for box in unit:
            if len(values[box]) == 2:
                for peer in unit:
                    if peer != box and values[box] == values[peer]:
                        for peer2 in unit:
                            if peer2 != box and peer2 != peer:
                                for digit in values[box]:
                                    values[peer2] = values[peer2].replace(digit, '')
    return values

In [194]:
def reduce_puzzle(values):
    import copy
    """Reduce a Sudoku puzzle by repeatedly applying all constraint strategies

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict or False
        The values dictionary after continued application of the constraint strategies
        no longer produces any changes, or False if the puzzle is unsolvable 
        
    Notes
    -----
    complete this function using elimination and only choice startegies then extending it to call the 
    naked pairs strategy.
    """
    changed = True
    while changed:
        temp = copy.deepcopy(values)
        changed = False
        values = eliminate(values)
        values = only_choice(values)
        values = naked_pairs(values)
        for box in boxes:
            if len(temp[box]) != len(values[box]):
                changed = True
            if len(values[box]) == 0:
                return False
    return values

In [195]:
def search(values):
    import copy
    """Apply depth first search to solve Sudoku puzzles in order to solve puzzles
    that cannot be solved by repeated reduction alone.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict or False
        The values dictionary with all boxes assigned or False
    """
    temp = copy.deepcopy(values)
    temp = reduce_puzzle(temp)
    for box in values:
        if len(values[box]) > 1:
            for digit in values[box]:
                temp = copy.deepcopy(values)
                temp[box] = digit
                temp = reduce_puzzle(temp)
                if temp:
                    return temp
            return False
    return values

In [196]:
def solve(grid):
    """Find the solution to a Sudoku puzzle using search and constraint propagation

    Parameters
    ----------
    grid(string)
        a string representing a sudoku grid.
        
        Ex. '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'

    Returns
    -------
    dict or False
        The dictionary representation of the final sudoku grid or False if no solution exists.
    """
    values = grid_values(grid)
    values = search(values)
    return values

In [197]:
def main():
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    display(grid_values(diag_sudoku_grid))
    result = solve(diag_sudoku_grid)
    print('\n------------------------------------ Solution ---------------------------------------------\n')
    display(result)

In [198]:
main()

{'A1': '2', 'A2': '123456789', 'A3': '123456789', 'A4': '123456789', 'A5': '123456789', 'A6': '123456789', 'A7': '123456789', 'A8': '123456789', 'A9': '123456789', 'B1': '123456789', 'B2': '123456789', 'B3': '123456789', 'B4': '123456789', 'B5': '123456789', 'B6': '6', 'B7': '2', 'B8': '123456789', 'B9': '123456789', 'C1': '123456789', 'C2': '123456789', 'C3': '1', 'C4': '123456789', 'C5': '123456789', 'C6': '123456789', 'C7': '123456789', 'C8': '7', 'C9': '123456789', 'D1': '123456789', 'D2': '123456789', 'D3': '6', 'D4': '123456789', 'D5': '123456789', 'D6': '8', 'D7': '123456789', 'D8': '123456789', 'D9': '123456789', 'E1': '3', 'E2': '123456789', 'E3': '123456789', 'E4': '123456789', 'E5': '9', 'E6': '123456789', 'E7': '123456789', 'E8': '123456789', 'E9': '7', 'F1': '123456789', 'F2': '123456789', 'F3': '123456789', 'F4': '6', 'F5': '123456789', 'F6': '123456789', 'F7': '4', 'F8': '123456789', 'F9': '123456789', 'G1': '123456789', 'G2': '4', 'G3': '123456789', 'G4': '123456789', '